# CashLog Advanced Analysis

In reality, CashLogs decision problem is more complex. If we assume that capacities of a cash center can be increased/reduced, we have to also consider that the annual costs of a cash center depend on its capacity/its volume. It is resonable to assume that:
- there is an annual fixed cost per cash center (e.g. for overheads, rent, insurance). However, this will also depend on the capacity, but will not scale proportionally with the volume (i.e., there are economies of scale)
- there are annual costs per cash center (e.g. for labor required to process the volume) that are dependent on the volume. They will likely also not be constant, i.e., there will also be economies of scale so that a very large cash center has lower labor costs per delivery than a small cash center

How can we capture this cost structure? 
- Let’s assume there are f types of cash centers: very small (v), small (s), medium (m), large (l), huge (h)
- Each cash center has fixed and variable costs depending on whether it is a small, medium, large or huge center

We will use this notebook to implement an advanced version of CashLog's decision problem by performing the following steps:
1. Define and load relevent model parameters
2. Define and initialize the decision variables
3. Define and implement the objective function
3. Define and implement the relevant constraints
4. Solve the problem and anlyse the results
5. Perform sensitivity analysis

### Load the required libraries

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import folium
from tqdm import tqdm
from pulp import *

%matplotlib inline

### The decision problem can be modeled in the sense of a MIP

In [ ]:
prob = LpProblem('CashLog_AdvancedAnalysis', LpMinimize)

### Define and load model parameters
$T = \{v,s,m,l,h\}$ Set of possible types of cash centers (very small ($v$), small ($s$), medium ($m$), large ($l$), or huge ($h$))

$d_j:$ Deliveries required in region $j$<br>
$V^{lb}_t:$ Lower bound of the volume (deliveries) for each type of cash center $t$<br>
$V^{ub}_t:$ Upper bound of the volume (deliveries) for each type of cash center $t$<br>
$c^{fix}_t:$ Fixed annual costs of a cash center of type $t$<br>
$c^{var}_t:$ Variable costs per delivery of a center of type $t$<br>

In [ ]:
warehouses = pd.read_csv('data/warehouses.csv', index_col='warehouseID')
W = warehouses.index.values

regions = pd.read_csv('data/regions_advanced_analysis.csv', index_col='regionID')
R = regions.index.values

shifts = pd.read_csv('data/shifts.csv', index_col=['warehouseID', 'regionID'])
S = shifts.index.values

In [ ]:
fixCostFunction = {
    'v': {'lower_bound': 0, 'upper_bound': 19348, 'c_fix': 61165, 'c_var': 4.14},
    's': {'lower_bound': 19349, 'upper_bound': 45415, 'c_fix': 86071, 'c_var': 2.85},
    'm': {'lower_bound': 45416, 'upper_bound': 107327, 'c_fix': 145100, 'c_var': 1.55}, 
    'l': {'lower_bound': 107328, 'upper_bound': 199999, 'c_fix': 145100, 'c_var': 1.55}, 
    'h': {'lower_bound': 200000, 'upper_bound': 99999999, 'c_fix': 145100, 'c_var': 1.55}
}

T = fixCostFunction.keys()

In [ ]:
def calculate_fixed_costs(x):
    for c in fixCostFunction.keys():
        if x >= fixCostFunction[c]['lower_bound'] and x <= fixCostFunction[c]['upper_bound']:
            fixed = fixCostFunction[c]['c_fix']
            variable = fixCostFunction[c]['c_var'] * x
            total_fixed_costs = fixed + variable
            return fixed, variable, total_fixed_costs, c
        
total_costs = []
fixed_costs = []
variable_costs = []
warehouse_size = []
x_range = range(0, 300000, 10)
for x in x_range:
    fixed_costs_tmp, variable_costs_tmp, total_costs_tmp, warehouse_size_tmp = calculate_fixed_costs(x)
    fixed_costs.append(fixed_costs_tmp)
    variable_costs.append(variable_costs_tmp)
    total_costs.append(total_costs_tmp)
    warehouse_size.append(warehouse_size_tmp)
    
plot_df = pd.DataFrame({'fixed_costs': fixed_costs, 'variable_costs': variable_costs, 
                        'total_costs':total_costs, 'warehouse_size': warehouse_size, 'x': x_range})

plot_df.rename(columns={'fixed_costs': 'fixed', 'variable_costs': 'variable'}, inplace=True)
plot_df.plot.area(x='x', y=['fixed', 'variable'], alpha=0.3, figsize=(16,10), )
for t in fixCostFunction.keys():
    if t != 'h':
        plt.axvline(fixCostFunction[t]['upper_bound'], c='grey', linestyle='--')
        plt.text(x=fixCostFunction[t]['lower_bound'] + (fixCostFunction[t]['upper_bound']-fixCostFunction[t]['lower_bound'])/2, y=
                550000, s=t)
plt.text(x=250000, y=550000, s='h')
plt.xlim(0,300000)
plt.xlabel('Deliveries')
plt.ylabel('Total Fixed Costs')

In our first analysis we will use the current cost function I:

### Define and initialize the decision variables

To model the advanced version of the decision problem we need additional decision variables:

$x_{ij}:$ Binary variable indicating if region $j$ is served by cash center $i$<br> 
$y_{it}:$ Binary variable indicating if cash center $j$ is of type $t$; if the cash center is closed,, $y_{it}$ will be zero for all $t\in \{v,s,m,l,h\}$<br>
$z_{it}:$ is the number of deliveries of cash center $i$ of type $t$<br>

In [ ]:
x = LpVariable.dicts(name='x', indices=S, cat=LpBinary)
y = LpVariable.dicts(name='y', indices=(W,T), cat=LpBinary)
z = LpVariable.dicts(name='z', indices=(W,T), lowBound=0, upBound=9999999, cat=LpContinuous)

### Define and implement the objective function

We want to minimize the total network costs (cash center costs + service costs).

$$\min \sum_{i\in W} \sum_{j\in R} x_{ij} c_{ij} + \sum_{i\in W} \sum_{t\in T} c^{fix}_t y_{it} + \sum_{i\in W} \sum_{t\in T} c^{var}_t z_{it}$$

In [ ]:
variableCosts = lpSum([x[i,j] * shifts.loc[i,j].transportationCosts for i,j in S])

fixedCosts = lpSum([z[i][t] * fixCostFunction[t]['c_var'] for i in W for t in T]) + \
lpSum([y[i][t] * fixCostFunction[t]['c_fix'] for i in W for t in T])

prob += fixedCosts + variableCosts

### Define and implement the relevant constraints

Regions can only be served by open warehouses:<br>
$$x_{ij} \leq \sum_{t\in T} y_{it} \quad \forall i\in W, j\in R$$

Each region has to be served by exactly one warehouse:<br>
$$\sum_{i\in W} x_{ij} = 1 \quad \forall j\in R$$

Warehouses have to handle the entire volume of each assigned region:
$$\sum_{t\in T} z_{it} = \sum_{j\in R} x_{ij} d_j \quad \forall i\in W$$ 

Fixed cost categories have to be selected depending on the handled volume in a warehouse:
$$z_{it} \geq V^{lb}_t y_{it} \quad \forall i\in W, t\in T$$
$$z_{it} \leq V^{ub}_t y_{it} \quad \forall i\in W, t\in T$$

In [ ]:
for i in W:
    for j in R:
        prob += x[i,j] <= lpSum([y[i][t] for t in T]) 

for j in R:
    prob += lpSum([x[i,j] for i in W]) == 1

for i in W:
    prob += lpSum([z[i][t] for t in T]) == lpSum([x[i,j] * regions.loc[j].sumDeliveries for j in R])

for i in W:
    for t in T:
        prob += z[i][t] >= fixCostFunction[t]['lower_bound'] * y[i][t]
        prob += z[i][t] <= fixCostFunction[t]['upper_bound'] * y[i][t]

### Solve the problem and analyze the results

In [ ]:
status = prob.solve(PULP_CBC_CMD())

In [ ]:
# status = prob.solve(solver=PULP_CBC_CMD())

In [ ]:
variableCosts = sum([x[i,j].varValue * shifts.loc[i,j].transportationCosts for i,j in S])
cash_center_fixed = sum([y[i][t].varValue * fixCostFunction[t]['c_fix'] for i in W for t in T])
cash_center_var = sum([z[i][t].varValue * fixCostFunction[t]['c_var'] for i in W for t in T])
print('Minimal costs are {:0,.0f} Euro'.format(prob.objective.value()))
print('Service Costs: {:0,.0f}€\nCash center fixed costs: {:0,.0f}€\n Cash center variable costs: {:0,.0f}€'.format(variableCosts,
                                                                                             cash_center_fixed,
                                                                                             cash_center_var))
print('To minimize costs the following warehouses should be closed:')
for i in W:
    if sum([y[i][t].varValue for t in T]) < 0.1:
        print('   -{}'.format(warehouses.loc[i].city))

### Perform sensitivity analysis

To perform sensitivity analysis and get a deeper understanding of the optimal decisions we solve multiple instances of the problem. In order to have a clean notebook we outsourced the model and import it for the analysis.

We can instantiate an instance of the model via the ```CashLogWLPadvanced``` class and specify the costs per shift as well as the fixed cost function we want to analyse.

In [ ]:
from advancedModel import CashLogWLPadvanced

In [ ]:
wlp = CashLogWLPadvanced()
wlp.solve(n_warehouses=-1)
print('Minimal costs are {:0,.0f} Euro'.format(wlp.totalCosts))
print('Service Costs: {:0,.0f}€\nCash center fixed costs: {:0,.0f}€\n Cash center variable costs: {:0,.0f}€'.format(wlp.variableCosts,
                                                                                             wlp.cash_center_fixed,
                                                                                             wlp.cash_center_var))
print('To minimize costs the following warehouses should be closed:')
for i in wlp.warehouse_results:
    if i['open'] < 0.1:
        print('   -{}'.format(i['city']))

Again, we can visualize the results on a map:

In [ ]:
plot_df_regions = pd.DataFrame(wlp.region_results)
plot_df_warehouses = pd.DataFrame(wlp.warehouse_results)
plot_df_warehouses = plot_df_warehouses[plot_df_warehouses.open == 1]
palette = sns.color_palette(None, len(wlp.W)).as_hex()
palette = {wlp.W[i]: palette[i] for i in range(len(wlp.W))}

m = folium.Map(location=[41, -4], zoom_start=6)
plot_df_regions.apply(lambda row: folium.Circle(location=[row['lat'], row['lon']], 
                                                radius=4000, fill=False, popup=row['city'],
                                                color=palette[row['warehouseID']]).add_to(m), axis=1)
plot_df_warehouses.apply(lambda row: folium.Circle(location=[row['lat'], row['lon']], 
                                                   radius=10000, fill=True, popup=row['city'],
                                                   color=palette[row['warehouseID']], 
                                                   fill_opacity=1).add_to(m), axis=1)
m

And compute optimal decisions for different settings:

In [ ]:
cash_center_fixed = {}
cash_center_var = {}

cVariable = {}
cTotal = {}
warehouse_results = {}
region_results = {}

wlp = CashLogWLPadvanced()

for c in [1]:
    cash_center_fixed[c] = {}
    cash_center_var[c] = {}
    cVariable[c] = {}
    cTotal[c] = {}
    warehouse_results[c] = {}
    region_results[c] = {}
    for n in tqdm(range(12, len(wlp.W)+1, 1)):
        wlp = CashLogWLPadvanced()
        wlp.solve(n_warehouses=n, fixCostFunction=c)
        cash_center_fixed[c][n] = wlp.cash_center_fixed
        cash_center_var[c][n] = wlp.cash_center_var
        cVariable[c][n] = wlp.variableCosts
        cTotal[c][n] = wlp.totalCosts
        warehouse_results[c][n] = wlp.warehouse_results
        region_results[c][n] = wlp.region_results

In [ ]:
results_df = pd.DataFrame({'cash_center_fixed':cash_center_fixed[1], 
                           'cash_center_var': cash_center_var[1],
                           'transportation':cVariable[1], 
                           'total':cTotal[1]})
results_df['n_warehouses'] = results_df.index.values
results_df['cost_function'] = 1

plot_df = pd.melt(results_df, id_vars=['n_warehouses', 'cost_function'], 
                  value_vars=['cash_center_fixed','cash_center_var',
                              'transportation', 'total'])


def millions(x, pos):
    return '%1.1fM' % (x * 1e-6)


def plot_costs(fixCostFunction):
    tmp = plot_df.loc[plot_df.cost_function ==fixCostFunction]
    min_costs_n = tmp.loc[tmp.loc[tmp.variable == 'total'].value.idxmin()].n_warehouses
    formatter = FuncFormatter(millions)
    tmp.replace(['cash_center_fixed', 'cash_center_var', 
                 'transportation', 'total'], ['Warehouse fix', 'Warehouse var',
                                              'Transportation', 'Total'], inplace=True)
    sns.set(font_scale=1.6)
    sns.set_style(style='white')
    fig, ax = plt.subplots(figsize=(12, 12))
    g = sns.lineplot(data=tmp, x='n_warehouses', y='value', 
                     hue='variable', palette={'Warehouse fix': 'darkblue',
                                              'Warehouse var': 'blue', 
                                              'Transportation': 'green', 
                                              'Total': 'red'})

    g.axvline(min_costs_n, c='grey', linestyle='--')
    ax.set_ylim(0,200000000)
    ax.yaxis.set_major_formatter(formatter)
    ax.set_xlabel('Number of Warehouses')
    ax.set_ylabel("Costs")
    ax.legend(bbox_to_anchor=[0.5,1.03], loc='center', ncol=4, frameon=False)
    plt.show()

In [ ]:
plot_costs(1)

In [ ]:
plot_df_2 = results_df.copy()
plot_df_2['cash_center'] = plot_df_2['cash_center_fixed'] + plot_df_2['cash_center_var']
min_costs_n = plot_df_2.loc[plot_df_2.total.idxmin()].n_warehouses
plot_df_2 = pd.melt(plot_df_2[['n_warehouses', 'transportation', 'cash_center', 'total']], id_vars=['n_warehouses'], 
                  value_vars=['transportation', 'cash_center', 'total'])

formatter = FuncFormatter(millions)
plot_df_2.replace(['cash_center', 'transportation', 'total'], ['Warehouse', 'Transportation', 'Total'], inplace=True)
sns.set(font_scale=1.6)
sns.set_style(style='white')
fig, ax = plt.subplots(figsize=(12, 12))
g = sns.lineplot(data=plot_df_2, x='n_warehouses', y='value', hue='variable', palette={'Warehouse': 'blue', 
                                                                              'Transportation': 'green', 
                                                                              'Total': 'red'})
g.axvline(min_costs_n, c='grey', linestyle='--')
ax.set_ylim(0,200000000)
ax.yaxis.set_major_formatter(formatter)
ax.set_xlabel('Number of Warehouses')
ax.set_ylabel("Costs")
ax.legend(bbox_to_anchor=[0.5,1.03], loc='center', ncol=3, frameon=False)

In [ ]:
def map_results(n_warehouses, fixCostFunction=1):
    plot_df_regions = pd.DataFrame(region_results[fixCostFunction][n_warehouses])
    plot_df_warehouses = pd.DataFrame(warehouse_results[fixCostFunction][n_warehouses])
    plot_df_warehouses = plot_df_warehouses[plot_df_warehouses.open == 1]
    palette = sns.color_palette(None, len(wlp.W)).as_hex()
    palette = {wlp.W[i]: palette[i] for i in range(len(wlp.W))}

    m = folium.Map(location=[41, -4], zoom_start=6)
    plot_df_regions.apply(lambda row: folium.Circle(location=[row['lat'], row['lon']], 
                                                    radius=4000, fill=False, popup=row['city'],
                                                    color=palette[row['warehouseID']]).add_to(m), axis=1)
    plot_df_warehouses.apply(lambda row: folium.Circle(location=[row['lat'], row['lon']], 
                                                       radius=10000, fill=True, popup=row['city'],
                                                       color=palette[row['warehouseID']], 
                                                       fill_opacity=1).add_to(m), axis=1)
    return m

In [ ]:
map_results(18)

In [ ]:
map_results(39)